# Zomata Stock Price Predicting using Time-Series analysis


## GRU Model

GRU (Gated Recurrent Unit) is a recurrent neural network (RNN) that processes sequential input, such as time series or natural language processing. GRUs, like LSTMs, are intended to overcome the vanishing gradient problem that can arise while training standard RNNs. However, GRUs are simpler and have fewer parameters than LSTMs, making them quicker to train and less susceptible to overfitting.

The GRU layer is imported from the tensorflow.keras.layers module, which is a component of the TensorFlow deep learning system. The GRU layer is used to generate a GRU model that can be trained using sequential data, such as stock price data. The GRU layer accepts an input shape, which determines the amount of time steps and features in the input data, and returns a series of hidden states that can then fed into a dense layer to make predictions.

## ARCH Model

An ARCH (Autoregressive Conditional Heteroskedasticity) model is a statistical method for modelling and forecasting time series data with time-varying volatility. Robert F. Engle introduced the ARCH model in 1982. The ARCH model is a form of GARCH (Generalised Autoregressive Conditional Heteroskedasticity) model, which is a broader family of models that enable the variance to be determined by previous variances and errors over a longer period of time. The ARCH model is used to calculate the variance of a time series as a function of previous variances and errors. The model is trained via maximum likelihood estimation (MLE) or another estimate approach.The ARCH model is used to model and forecast time series data that has variable volatility over time, such as stock prices, exchange rates, and interest rates. The ARCH model is a statistical model that is trained with the arch_model class in the arch Python package. The model is trained on previous time series data and then used to anticipate future data.



## GRU vs ARCH 

GRU (Gated Recurrent Unit) models and ARCH (Autoregressive Conditional Heteroskedasticity) models are both used for time series forecasting, although they operate on distinct principles and with different types of data.

GRU models are recurrent neural networks (RNNs) that analyse sequential data, such as time series or natural language processing. GRU models forecast future stock values using previous data.

ARCH models, on the other hand, are statistical models for modelling and forecasting time series data with variable volatility across time. The ARCH model is used to calculate the variance of a time series as a function of previous variances and errors.

When it comes to forecasting stocks, GRU models offer some benefits over ARCH models:

- GRU models are more adaptable and can capture more complicated patterns in data than ARCH models.
- GRU models can accommodate missing and unevenly spaced data, but ARCH models require consistently spaced data.
- GRU models may anticipate many steps ahead, whereas ARCH models are commonly employed for one-step forecasting.
- GRU models can handle both univariate and multivariate time series data, whereas ARCH models are mainly utilised with univariate data.


In conclusion, GRU models are more flexible and capable of capturing more complicated patterns in data than ARCH models. As a result, GRU models outperform ARCH models in terms of stock prediction.


## Code Breakdown

### Data Preperation(prep_data_function)

In [ ]:
# Function to prep csv file
def prepare_data():
    # Read stock price data
    url = 'https://raw.githubusercontent.com/Tiaan-Botes/CYO_Project_Group_E/52663ba4e6833e232f1bbd7a0ab48edb23f52b91/data/data.csv'
    data = pd.read_csv(url)

    data['Date'] = pd.to_datetime(data['Date'])
    data.sort_values('Date', inplace=True)
    data.set_index('Date', inplace=True)
    data.drop(columns=['Unnamed: 0', 'Year', 'Month', 'Day', 'Weekday'], inplace=True)

    data['ma_30'] = data['Adj Close'].rolling(window=30).mean()
    data['ma_90'] = data['Adj Close'].rolling(window=90).mean()
    
    data['daily_returns'] = data['Adj Close'].pct_change()*100
    data.dropna(inplace=True)

    return data

The programme begins by retrieving stock price information from a CSV file available on GitHub. This information most likely includes historical stock values for a certain firm.

It then processes the data.
- Converts the 'Date' column to datetime format and uses it as an index. This step makes the date the main identifier for each data entry.
- Sorts the data by date to maintain chronological order.
- Removes unnecessary columns like 'Unnamed: 0', 'Year', 'Month', 'Day', and 'Weekday'. These columns can be superfluous or irrelevant to the study.
- For the 'Adj Close' column, two moving averages (MA) are calculated across various time frames (30 and 90 days). Moving averages are often used to smooth out volatility and show long-term patterns in data.
- Calculates daily returns using the percentage change in the 'Adj Close' column from one day to the next. This measure gives insight into the stock's daily volatility or performance.
- Removes rows with missing values to ensure data integrity.




## Visualization






In [ ]:
df = prepare_data()
df.info()
print(df.head())

plot_data = df.loc['2023-01-01':'2023-12-31']

# Plotting stock price along with moving averages
plt.figure(figsize=(20, 25))
fig = px.line(
    data_frame=plot_data, 
    x=plot_data.index, 
    y=['Adj Close', 'ma_30', 'ma_90']
)
fig.show()

# Plotting daily returns
plt.figure(figsize=(10, 25))
fig = px.line(
    data_frame=df, 
    x=df.index, 
    y=['daily_returns']
)
fig.show()


After preprocessing the data, the code creates visualisations to gather insights:
- The first chart depicts the stock's adjusted closing price, as well as its 30-day and 90-day moving averages. This visualisation depicts the trend and fluctuation of the stock price over time.
- The second plot depicts the stock's daily returns, allowing analysts to discover times of significant volatility or unexpected price moves.

## GRU Model Preparation and Training:

In [ ]:
# Function to prepare data for GRU model
def prepare_gru_data():
    data = prepare_data()  
    dataset = data[['Adj Close']].values.astype('float32')
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)

    return dataset

# Function to build GRU model
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(units=50, input_shape=input_shape))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

# Function to train GRU model
def train_gru_model(dataset):
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

    def create_dataset(dataset, look_back=1):
        X, Y = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            X.append(a)
            Y.append(dataset[i + look_back, 0])

        return np.array(X), np.array(Y)

    look_back = 1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

    model = build_gru_model((1, look_back))
    model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

    return model, testX, testY
# Function to make predictions using GRU model
def predict_gru_model(model, testX):
    return model.predict(testX)

# Preparing data
gru_dataset = prepare_gru_data()

# Training 
gru_model, testX, testY = train_gru_model(gru_dataset)

# predictions
gru_predictions = predict_gru_model(gru_model, testX)

The algorithm then prepares data for training a Gated Recurrent Unit (GRU) model, 
The prepare_gru_data method pulls the 'Adj Close' column from the generated DataFrame and scales it using MinMaxScaler, transforming the data into a range of 0 to 1. Scaling guarantees that all input characteristics contribute equally to model training while preventing any one element from overpowering the others.


Next, the code specifies functions for constructing, training, and utilising the GRU model.
The build_gru_model method creates a Sequential Neural Network model with TensorFlow's Keras API. It is made up of a GRU layer with 50 units, followed by a Dense layer with one unit. The model is built using the Adam optimizer and mean squared error loss, both of which are popular methods for training regression models.

The train_gru_model function divides the dataset into training and testing sets, generates input-output pairs by shifting the time series data with a defined look-back window, reshapes the input data to fit the GRU model's input criteria, constructs the model, and trains it on the training data. The model is trained for 100 epochs(pass throughs), with a batch size of one.

The predict_gru_model function uses the trained GRU model and input data to make predictions for the test set.


## Model Assessment:

Finally, the algorithm assesses the performance of the GRU model and displays the outcomes:

It estimates the Root Mean Squared Error (RMSE) of the actual and anticipated values. The RMSE is a measure of the model's prediction accuracy that shows the average size of the mistakes.

The model's performance is graphically compared by plotting the actual and projected numbers. This enables analysts to determine how well the model reflects the data's underlying trends and if it can accurately forecast future stock values.
